In [75]:
from openai import OpenAI
from docx import Document
from docx.shared import Pt, Inches, RGBColor
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.enum.style import WD_STYLE_TYPE
from docx.oxml import OxmlElement, parse_xml
from docx.oxml.ns import qn
from datetime import date

In [76]:
# Set up OpenAI API key
api_key = ''
client = OpenAI(api_key=api_key)

In [77]:
def generate_index(project_requirements):
    prompt = f"""
    Based on the following project requirements, generate a structured index for a Statement of Work document.
    The index should be in the format:
    1. Main topic
        a. Subtopic
            1. Sub-subtopic

    Project requirements: {project_requirements}

    Generate the index:
    """

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that generates structured indexes for Statement of Work documents."},
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content

In [78]:
def generate_content_for_topic(topic):
    prompt = f"""
    Generate detailed content for the following topic in a Statement of Work document:

    {topic}

    Provide comprehensive and professional content for this section:
    """

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that generates detailed content for Statement of Work documents."},
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content

In [79]:
def parse_index(index_content):
    lines = index_content.strip().split('\n')
    toc = []
    for line in lines:
        if line.strip():
            level = 1
            if line.startswith('    '):
                level = 3
            elif line.startswith('  '):
                level = 2
            toc.append((line.strip(), level))
    return toc

In [80]:
def create_custom_style(document, style_name, font_name, font_size, font_color, bold=False, italic=False):
    styles = document.styles
    custom_style = styles.add_style(style_name, WD_STYLE_TYPE.PARAGRAPH)
    custom_style.font.name = font_name
    custom_style.font.size = Pt(font_size)
    custom_style.font.color.rgb = RGBColor(*font_color)
    custom_style.font.bold = bold
    custom_style.font.italic = italic
    return custom_style

In [81]:
def create_custom_styles(doc):
    create_custom_style(doc, 'Title Style', 'Arial', 24, (0, 0, 139), bold=True)  # Dark Blue
    create_custom_style(doc, 'Heading1 Style', 'Arial', 18, (0, 0, 0), bold=True)
    create_custom_style(doc, 'Heading2 Style', 'Arial', 16, (68, 68, 68), bold=True)
    create_custom_style(doc, 'Heading3 Style', 'Arial', 14, (102, 102, 102), bold=True)

In [82]:
def setup_header(section):
    header = section.header
    header_para = header.paragraphs[0]
    header_para.alignment = WD_ALIGN_PARAGRAPH.RIGHT
    current_date = date.today().strftime("%d/%m/%Y")
    date_run = header_para.add_run(f'Date: {current_date}')
    date_run.font.color.rgb = RGBColor(183, 183, 183)
    date_run.font.name = 'Arial'
    date_run.font.size = Pt(10)

In [83]:
def add_page_number(run):
    fldChar = OxmlElement('w:fldChar')
    fldChar.set(qn('w:fldCharType'), 'begin')
    instrText = OxmlElement('w:instrText')
    instrText.text = "PAGE"
    fldChar2 = OxmlElement('w:fldChar')
    fldChar2.set(qn('w:fldCharType'), 'end')
    
    run._element.append(fldChar)
    run._element.append(instrText)
    run._element.append(fldChar2)

In [84]:
def add_page_number_to_footer(footer):
    page_num_para = footer.add_paragraph()
    page_num_para.alignment = WD_ALIGN_PARAGRAPH.RIGHT
    page_num_run = page_num_para.add_run('Page ')
    page_num_run.font.color.rgb = RGBColor(183, 183, 183)
    page_num_run.font.name = 'Arial'
    page_num_run.font.size = Pt(10)
    add_page_number(page_num_run)

In [85]:
def add_company_name_to_footer(footer, company_name):
    company_para = footer.add_paragraph()
    company_para.alignment = WD_ALIGN_PARAGRAPH.RIGHT
    company_run = company_para.add_run(company_name)
    company_run.font.color.rgb = RGBColor(183, 183, 183)
    company_run.font.name = 'Arial'
    company_run.font.size = Pt(10)

In [86]:
def setup_footer(section, company_name):
    footer = section.footer
    add_page_number_to_footer(footer)
    add_company_name_to_footer(footer, company_name)

In [87]:
def setup_header_and_footer(doc, company_name):
    section = doc.sections[0]
    setup_header(section)
    setup_footer(section, company_name)

In [88]:
def create_title_page(doc, project_name, version, company_name):
    title = doc.add_paragraph('Statement of Work', style='Title Style')
    title.alignment = WD_ALIGN_PARAGRAPH.CENTER
    doc.add_paragraph()
    doc.add_paragraph(f"Project Name: {project_name}")
    doc.add_paragraph(f"Version: {version}")
    doc.add_paragraph(f"Date: {date.today().strftime('%d/%m/%Y')}")
    doc.add_paragraph(f"Company: {company_name}")
    doc.add_page_break()

In [89]:
def create_table_of_contents(doc, toc):
    doc.add_paragraph('Table of Contents', style='Heading1 Style')
    for item, level in toc:
        p = doc.add_paragraph()
        p.paragraph_format.left_indent = Pt((level - 1) * 12)  # Adjust indentation
        style = f'Heading{level} Style' if level <= 3 else 'Heading3 Style'
        p.style = doc.styles[style]
        p.add_run(item)
    doc.add_page_break()

In [90]:
def set_run_background_color(run, r, g, b):
    rPr = run._element.get_or_add_rPr()
    shd = OxmlElement('w:shd')
    shd.set(qn('w:val'), 'clear')
    shd.set(qn('w:color'), 'auto')
    shd.set(qn('w:fill'), '{:02x}{:02x}{:02x}'.format(r, g, b))
    rPr.append(shd)

def create_content_pages(doc, toc):
    for item, level in toc:
        style = f'Heading{level} Style' if level <= 3 else 'Heading3 Style'
        p = doc.add_paragraph(style=style)
        run = p.add_run(item)
        set_run_background_color(run, 204, 204, 204)  # Set background color to RGB(204, 204, 204)
        p.paragraph_format.space_after = Pt(12)  # Add some space after the heading
        
        content = generate_content_for_topic(item)
        doc.add_paragraph(content)
        
        if level == 1:
            doc.add_page_break()

In [91]:
def create_sow_document(project_name, version, toc, company_name):
    doc = Document()
    create_custom_styles(doc)
    setup_header_and_footer(doc, company_name)
    create_title_page(doc, project_name, version, company_name)
    create_table_of_contents(doc, toc)
    create_content_pages(doc, toc)
    doc.save('sow_document.docx')

In [92]:
def main():
    project_name = "Oxford Mtrain"
    version = "version"
    company_name = "© Techuz Infoweb Pvt. Ltd."
    project_requirements = """The objective of this document is to define the features of an Oxford Mtrain Web Application development, which aims to develop a training platform for the medical therapist. Its primary objective is to provide a user-friendly and secure interface for therapists to get the training using a chatbot. Platform offers a meticulously designed user journey aimed at providing a seamless and enriching experience. Beginning with a straightforward Login/Signup process, users embark on a path that not only involves onboarding to complete their profiles and email verification but also necessitates the thoughtful purchase of subscription plans tailored to their training needs. As users delve into the system, they can access and practice a curated list of exercises, engaging in role plays with our AI chatbot patients. The platform's versatility allows users to switch roles between client and supervisor, fostering an intuitive and dynamic learning environment. The ability to switch between voice and text communication adds another layer of adaptability. Moreover, users can meticulously track their progress, receive scores from ChatGPT supervisors, and contribute valuable feedback to enhance the app's efficacy. With features like exporting conversations, seamless roleplays with ChatGPT supervisors, and robust profile management, our platform endeavors to provide therapists with a comprehensive and empowering journey for skill development and continuous improvement."""

    print("Generating index...")
    index_content = generate_index(project_requirements)
    toc = parse_index(index_content)
    
    print("Creating document and generating content...")
    create_sow_document(project_name, version, toc, company_name)
    print("SOW document has been generated and saved as 'sow_document.docx'")

In [93]:
main()

Generating index...
Creating document and generating content...
SOW document has been generated and saved as 'sow_document.docx'
